# Supervisor (Polyglot)

In [ ]:
#!import ../nbs/Testing.dib

In [ ]:
#r @"../../../../../../../.nuget/packages/fsharp.control.asyncseq/3.2.1/lib/netstandard2.1/FSharp.Control.AsyncSeq.dll"
#r @"../../../../../../../.nuget/packages/system.reactive/6.0.1-preview.1/lib/net6.0/System.Reactive.dll"
#r @"../../../../../../../.nuget/packages/system.reactive.linq/6.0.1-preview.1/lib/netstandard2.0/System.Reactive.Linq.dll"
#r @"../../../../../../../.nuget/packages/argu/6.1.1/lib/netstandard2.0/Argu.dll"
#r @"../../../../../../../.nuget/packages/system.commandline/2.0.0-beta4.22272.1/lib/net6.0/System.CommandLine.dll"
#r @"../../../../../../../.nuget/packages/asyncio/0.1.69/lib/netstandard2.0/AsyncIO.dll"
#r @"../../../../../../../.nuget/packages/netmq/4.0.1.13/lib/netstandard2.1/NetMQ.dll"
#r @"../../../../../../../.nuget/packages/fsharp.json/0.4.1/lib/netstandard2.0/FSharp.Json.dll"

In [ ]:
#!import ../nbs/Common.fs
#!import ../nbs/CommonFSharp.fs
#!import ../nbs/Async.fs
#!import ../nbs/AsyncSeq.fs
#!import ../nbs/Networking.fs
#!import ../nbs/Runtime.fs
#!import ../nbs/FileSystem.fs

In [ ]:
open Common
open FileSystem

## sendJson

In [ ]:
let inline sendJson (port : int) (json : string) = async {
    let! portOpen = Networking.testPortOpen port
    if portOpen then
        try
            // use runtime = new NetMQ.NetMQRuntime ()
            use request = new NetMQ.Sockets.RequestSocket ()
            request.Connect $"tcp://127.0.0.1:{port}"

            let msg = NetMQ.NetMQMessage ()
            msg.Append json

            NetMQ.OutgoingSocketExtensions.SendMultipartMessage (request, msg)
            let result = NetMQ.ReceivingSocketExtensions.ReceiveMultipartMessage (request, 10)
            // let! result = NetMQ.AsyncReceiveExtensions.ReceiveMultipartMessageAsync (request, 10) |> Async.AwaitTask
            let resultString = result |> Seq.map (fun x -> x.ConvertToString ()) |> String.concat ""
            trace Debug (fun () -> $"sendJson / port: {port} / json: {json} / result.FrameCount: {result.FrameCount} / resultString.Length: {resultString.Length}") getLocals
            return Some resultString
        with ex ->
            trace Critical (fun () -> $"sendJson / port: {port} / json: {json} / ex: {ex |> printException}") getLocals
            return None
    else
        trace Debug (fun () -> "sendJson / error: port not open") getLocals
        return None
}

## sendObj

In [ ]:
let inline sendObj port obj =
    obj
    |> System.Text.Json.JsonSerializer.Serialize
    |> sendJson port

## awaitCompiler

In [ ]:
type VSCPos = {| line : int; character : int |}
type VSCRange = VSCPos * VSCPos
type RString = VSCRange * string
type TracedError = {| trace : string list; message : string |}
type ClientErrorsRes =
    | FatalError of string
    | TracedError of TracedError
    | PackageErrors of {| uri : string; errors : RString list |}
    | TokenizerErrors of {| uri : string; errors : RString list |}
    | ParserErrors of {| uri : string; errors : RString list |}
    | TypeErrors of {| uri : string; errors : RString list |}

In [ ]:
let inline awaitCompiler port cancellationToken = async {
    let! ct =
        cancellationToken
        |> Option.defaultValue System.Threading.CancellationToken.None
        |> Async.mergeCancellationTokenWithDefaultAsync

    let compiler = MailboxProcessor.Start (fun inbox -> async {
        let! availablePort = Networking.getAvailablePort (Some 60) port
        if port = 13805 && availablePort <> port then
            let pingObj = {| Ping = true |}
            let! pingResult = pingObj |> sendObj port
            inbox.Post port
        else
            let repositoryRoot = FileSystem.getSourceDirectory () |> FileSystem.findParent ".paket" false

            let compilerPath =
                repositoryRoot </> "deps/The-Spiral-Language/The Spiral Language 2/artifacts/bin/The Spiral Language 2/release"
                |> System.IO.Path.GetFullPath

            let dllPath = compilerPath </> "Spiral.dll"
            // let commandsPath = compilerPath </> "compiler/supervisor/commands"

            let! exitCode, result =
                Runtime.executeWithOptionsAsync
                    {
                        Command = $@"dotnet ""{dllPath}"" port={availablePort}"
                        CancellationToken = Some ct
                        WorkingDirectory = None
                        OnLine = Some <| fun { Line = line } -> async {
                            if line |> String.contains $"Server bound to: tcp://*:{availablePort}"
                            then inbox.Post availablePort
                        }
                    }
            trace Debug (fun () -> $"startSupervisor / exitCode: {exitCode} / result: {result}") getLocals
    }, ct)

    let! serverPort = compiler.Receive ()

    let request = new NetMQ.Sockets.SubscriberSocket ()
    request.SubscribeToAnyTopic ()
    request.Connect $"tcp://127.0.0.1:{serverPort + 1}"

    let rec loop _i = async {
        try
            let result = NetMQ.ReceivingSocketExtensions.ReceiveMultipartMessage (request, 10)
            // let! result = NetMQ.AsyncReceiveExtensions.ReceiveMultipartMessageAsync (request, 10) |> Async.AwaitTask
            return
                result
                |> Seq.map (fun x -> x.ConvertToString ())
                |> String.concat ""
                |> FSharp.Json.Json.deserialize<ClientErrorsRes>
                |> Some
        with ex ->
            trace Critical (fun () -> $"awaitCompiler / ex: {ex |> printException}") getLocals
            return None
    }

    return
        serverPort,
        loop |> FSharp.Control.AsyncSeq.initInfiniteAsync |> FSharp.Control.AsyncSeq.choose id,
        (request :> System.IDisposable)
}

## getFileUri

In [ ]:
let inline getFileUri path =
    $"file:///{path |> String.trimStart [| '/' |]}"

In [ ]:
let inline getFilePathFromUri uri =
    match System.Uri.TryCreate (uri, System.UriKind.Absolute) with
    | true, uri -> uri.AbsolutePath |> System.IO.Path.GetFullPath
    | _ -> failwith "invalid uri"

## buildFile

In [ ]:
let inline buildFile timeout cancellationToken path = async {
    let fullPath = path |> System.IO.Path.GetFullPath
    let fileDir = fullPath |> System.IO.Path.GetDirectoryName
    let fileName = fullPath |> System.IO.Path.GetFileNameWithoutExtension
    let! code = fullPath |> FileSystem.readAllTextAsync

    let stream, disposable = FileSystem.watchDirectory true fileDir
    use _ = disposable

    let port =
        if fullPath |> String.startsWith (System.IO.Path.GetTempPath ())
            && fullPath |> String.contains "Microsoft.DotNet.Interactive.App" |> not
        then 13807
        else 13805

    let! serverPort, errors, disposable = awaitCompiler port cancellationToken
    use _ = disposable

    let fsxContentSeq =
        stream
        |> FSharp.Control.AsyncSeq.choose (function
            | _, FileSystemChange.Changed (path, Some content) when path = $"{fileName}.fsx" -> Some content
            | _ -> None
        )
        |> FSharp.Control.AsyncSeq.map (fun content ->
            Some (content |> String.replace "\r\n" "\n"), None
        )

    let inline printErrorData (data : {| uri : string; errors : RString list |}) =
        let fileName = data.uri |> System.IO.Path.GetFileName
        let errors =
            data.errors
            |> List.map snd
            |> String.concat "\n"
        $"{fileName}:\n{errors}"

    let errorsSeq =
        errors
        |> FSharp.Control.AsyncSeq.choose (fun error ->
            match error with
            | FatalError message ->
                Some (message, true, error)
            | TracedError data ->
                Some (data.message, true, error)
            | PackageErrors data when data.errors |> List.isEmpty |> not ->
                Some (data |> printErrorData, true, error)
            | TokenizerErrors data when data.errors |> List.isEmpty |> not ->
                Some (data |> printErrorData, true, error)
            | ParserErrors data when data.errors |> List.isEmpty |> not ->
                Some (data |> printErrorData, false, error)
            | TypeErrors data when data.errors |> List.isEmpty |> not ->
                Some (data |> printErrorData, true, error)
            | _ -> None
        )
        |> FSharp.Control.AsyncSeq.map (fun (message, fatal, error) ->
            None, Some (message, fatal, error)
        )

    let outputSeq =
        [fsxContentSeq; errorsSeq]
        |> FSharp.Control.AsyncSeq.mergeAll

    let! outputChild =
        ((None, [], None), outputSeq)
        ||> FSharp.Control.AsyncSeq.scan (
            fun (fsxContentResult, errors, firstErrorTicks) (fsxContent, error) ->
                match fsxContent, error with
                | Some fsxContent, None -> Some fsxContent, errors, firstErrorTicks
                | None, Some (message, fatal, error) ->
                    fsxContentResult,
                    (message, error) :: errors,
                    firstErrorTicks
                    |> Option.defaultWith (fun () ->
                        if fatal
                        then System.DateTime.MinValue.Ticks
                        else System.DateTime.Now.Ticks
                    )
                    |> Some
                | _ -> fsxContentResult, errors, firstErrorTicks
        )
        |> FSharp.Control.AsyncSeq.takeWhileInclusive (fun (fsxContent, errors, firstErrorTicks) ->
            let firstErrorMs =
                firstErrorTicks
                |> Option.map (fun t -> System.TimeSpan(System.DateTime.Now.Ticks - t).TotalMilliseconds)
            trace Debug (fun () -> $"buildFile / fsxContent: {fsxContent} / errors: {errors} / firstErrorMs: {firstErrorMs}") getLocals
            match fsxContent, errors, firstErrorMs with
            | None, [], _ -> true
            | None, _, Some firstErrorMs when firstErrorMs < 2000. -> true
            | _ -> false
        )
        |> FSharp.Control.AsyncSeq.map (fun (fsxContent, errors, _) ->
            fsxContent, errors |> List.distinct
        )
        |> FSharp.Control.AsyncSeq.tryLast
        |> Async.catch
        |> Async.runWithTimeoutAsync timeout
        |> Async.StartChild

    let fileOpenObj = {| FileOpen = {| uri = fullPath |> getFileUri; spiText = code |} |}
    let! _fileOpenResult = fileOpenObj |> sendObj serverPort

    let buildFileObj = {| BuildFile = {| uri = fullPath |> getFileUri; backend = "Fsharp" |} |}
    let! _buildFileResult = buildFileObj |> sendObj serverPort

    return!
        outputChild
        |> Async.map (function
            | Some (Ok (Some result)) -> result
            | Some (Error ex) ->
                trace Critical (fun () -> $"buildFile / error: {ex |> printException}") getLocals
                None, []
            | _ -> None, []
        )
}

## persistCode

In [ ]:
let inline persistCode timeout cancellationToken code = async {
    let tempDir = FileSystem.createTempDirectory ()

    let mainPath = tempDir </> "main.spi"
    do! code |> FileSystem.writeAllTextAsync mainPath

    let repositoryRoot = FileSystem.getSourceDirectory () |> FileSystem.findParent ".paket" false

    let spiprojPath = tempDir </> "package.spiproj"
    let spiprojCode =
        $"""packageDir: {repositoryRoot </> "spiral"}
packages:
    |core-
    fsharp
modules:
    main
"""
    do! spiprojCode |> FileSystem.writeAllTextAsync spiprojPath

    let disposable = newDisposable (fun () ->
        ()
        // tempDir |> FileSystem.deleteDirectoryAsync |> Async.Ignore |> Async.RunSynchronously
    )

    return mainPath, disposable
}

## buildCode

In [ ]:
let inline buildCode timeout cancellationToken code = async {
    let! mainPath, disposable = persistCode timeout cancellationToken code
    use _ = disposable
    return! mainPath |> buildFile timeout cancellationToken
}

In [ ]:
//// test

let buildCode timeout cancellationToken code = buildCode timeout cancellationToken code

In [ ]:
//// test

"inl app () =
    0i32

inl a = 1

inl main () =
    app
    |> dyn
    |> ignore
"
|> buildCode 8000 None
|> Async.runWithTimeout 8000
|> Option.map (fun (fsxContent, errors) -> fsxContent, errors |> List.map fst)
|> _equal (
    Some (
        Some "let rec closure0 () () : int32 =
    0
let v0 : (unit -> int32) = closure0()
()
",
        ["main.spi:
Global inl/let statements should all return functions known at parse time."]
    )
)

00:00:00 #1 [Debug] createTempDirectory / tempFolder: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230924-1117-5831-3146-38c627608304 / result: { CreationTime = 2023-09-24 11:17:58 AM
  Exists = true }
00:00:00 #2 [Debug] runWithTimeoutAsync / timeout: 60
00:00:00 #3 [Debug] executeAsync / options: { Command =
   "dotnet "C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" port=13807"
  WorkingDirectory = None
  CancellationToken = Some System.Threading.CancellationToken
  OnLine = Some <fun:buildCode@3-920> }
00:00:01 #4 [Debug] > Server bound to: tcp://*:13807 & tcp://*:13808
00:00:01 #5 [Debug] > pwd: C:\home\git\polyglot\apps\spiral
00:00:01 #6 [Debug] > dll_path: C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release
00:00:01 #7 [Debug] buildFile / fsxContent:  / errors: [] / firstErrorMs:
00:00:01 #8 [Debug] sendJson / port: 13807 / json: {"FileOpen

In [ ]:
//// test

"""inl app () =
    fsharp.console.write_line "error"
    1i32

inl main () =
    app
    |> dyn
    |> ignore
"""
|> buildCode 8000 None
|> Async.runWithTimeout 8000
|> Option.map (fun (fsxContent, errors) -> fsxContent, errors |> List.map fst)
|> _equal (
    Some (
        Some """let rec closure0 () () : int32 =
    let v0 : string = "error"
    System.Console.WriteLine v0
    1
let v0 : (unit -> int32) = closure0()
()
""",
        []
    )
)

00:00:03 #15 [Debug] createTempDirectory / tempFolder: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230924-1118-0229-2943-2a98c807f6a2 / result: { CreationTime = 2023-09-24 11:18:02 AM
  Exists = true }
00:00:04 #16 [Debug] runWithTimeoutAsync / timeout: 60
00:00:04 #17 [Debug] executeAsync / options: { Command =
   "dotnet "C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" port=13807"
  WorkingDirectory = None
  CancellationToken = Some System.Threading.CancellationToken
  OnLine = Some <fun:buildCode@3-920> }
00:00:04 #18 [Debug] > Server bound to: tcp://*:13807 & tcp://*:13808
00:00:04 #19 [Debug] buildFile / fsxContent:  / errors: [] / firstErrorMs:
00:00:04 #20 [Debug] > pwd: C:\home\git\polyglot\apps\spiral
00:00:04 #21 [Debug] > dll_path: C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release
00:00:04 #22 [Debug] sendJson / port: 13807 / json: {"

In [ ]:
//// test

""
|> buildCode 4000 None
|> Async.runWithTimeout 4000
|> _equal None

00:00:06 #28 [Debug] createTempDirectory / tempFolder: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230924-1118-0462-6226-64de8cc24436 / result: { CreationTime = 2023-09-24 11:18:04 AM
  Exists = true }
00:00:06 #29 [Debug] runWithTimeoutAsync / timeout: 60
00:00:06 #30 [Debug] executeAsync / options: { Command =
   "dotnet "C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" port=13807"
  WorkingDirectory = None
  CancellationToken = Some System.Threading.CancellationToken
  OnLine = Some <fun:buildCode@3-920> }
00:00:06 #31 [Debug] > Server bound to: tcp://*:13807 & tcp://*:13808
00:00:06 #32 [Debug] buildFile / fsxContent:  / errors: [] / firstErrorMs:
00:00:06 #33 [Debug] > pwd: C:\home\git\polyglot\apps\spiral
00:00:06 #34 [Debug] > dll_path: C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release
00:00:06 #35 [Debug] sendJson / port: 13807 / json: {"

In [ ]:
//// test

"""inl main () =
    1i32 / 0i32
"""
|> buildCode 8000 None
|> Async.runWithTimeout 8000
|> Option.map (fun (fsxContent, errors) -> fsxContent, errors |> List.map fst)
|> _equal (
    Some (
        None,
        ["An attempt to divide by zero has been detected at compile time."]
    )
)

00:00:10 #42 [Critical] awaitCompiler / ex: System.Net.Sockets.SocketException: An existing connection was forcibly closed by the remote host.
00:00:10 #43 [Debug] createTempDirectory / tempFolder: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230924-1118-0893-9392-952f8a19239d / result: { CreationTime = 2023-09-24 11:18:08 AM
  Exists = true }
00:00:10 #44 [Debug] runWithTimeoutAsync / timeout: 60
00:00:10 #45 [Debug] executeAsync / options: { Command =
   "dotnet "C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" port=13807"
  WorkingDirectory = None
  CancellationToken = Some System.Threading.CancellationToken
  OnLine = Some <fun:buildCode@3-920> }
00:00:11 #47 [Debug] > Server bound to: tcp://*:13807 & tcp://*:13808
00:00:11 #48 [Debug] buildFile / fsxContent:  / errors: [] / firstErrorMs:
00:00:11 #49 [Debug] > pwd: C:\home\git\polyglot\apps\spiral
00:00:11 #50 [Debug] > dll_path: C:\home\git\polyglot\de

In [ ]:
//// test

"""union a =
    | B
    | c

inl main () =
    ()
"""
|> buildCode 8000 None
|> Async.runWithTimeout 8000
|> Option.map (fun (fsxContent, errors) -> fsxContent, errors |> List.map fst)
|> _equal (
    Some (
        Some "()
",
        ["main.spi:
Expected: uppercase variable"]
    )
)

00:00:12 #56 [Debug] createTempDirectory / tempFolder: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230924-1118-1103-0346-0b665fb0eaa5 / result: { CreationTime = 2023-09-24 11:18:11 AM
  Exists = true }
00:00:12 #57 [Debug] runWithTimeoutAsync / timeout: 60
00:00:12 #58 [Debug] executeAsync / options: { Command =
   "dotnet "C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" port=13807"
  WorkingDirectory = None
  CancellationToken = Some System.Threading.CancellationToken
  OnLine = Some <fun:buildCode@3-920> }
00:00:13 #59 [Debug] > Server bound to: tcp://*:13807 & tcp://*:13808
00:00:13 #60 [Debug] buildFile / fsxContent:  / errors: [] / firstErrorMs:
00:00:13 #61 [Debug] > pwd: C:\home\git\polyglot\apps\spiral
00:00:13 #62 [Debug] > dll_path: C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release
00:00:13 #63 [Debug] sendJson / port: 13807 / json: {"

In [ ]:
//// test

"""
/// abc
inl main () =
    ()
"""
|> buildCode 8000 None
|> Async.runWithTimeout 8000
|> Option.map (fun (fsxContent, errors) -> fsxContent, errors |> List.map fst)
|> _equal (
    Some (
        None,
        ["main.spi:
Expected: whitespace"]
    )
)

00:00:14 #70 [Debug] createTempDirectory / tempFolder: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230924-1118-1318-1890-10e464584dea / result: { CreationTime = 2023-09-24 11:18:13 AM
  Exists = true }
00:00:14 #71 [Debug] runWithTimeoutAsync / timeout: 60
00:00:14 #72 [Debug] executeAsync / options: { Command =
   "dotnet "C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" port=13807"
  WorkingDirectory = None
  CancellationToken = Some System.Threading.CancellationToken
  OnLine = Some <fun:buildCode@3-920> }
00:00:15 #73 [Debug] > Server bound to: tcp://*:13807 & tcp://*:13808
00:00:15 #74 [Debug] buildFile / fsxContent:  / errors: [] / firstErrorMs:
00:00:15 #75 [Debug] > pwd: C:\home\git\polyglot\apps\spiral
00:00:15 #76 [Debug] > dll_path: C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release
00:00:15 #77 [Debug] sendJson / port: 13807 / json: {"

In [ ]:
//// test

"""
inl main () =
    real
        inl real_unbox forall a. (obj : a) : a =
            typecase obj with
            | _ => obj
        real_unbox ()
    ()
"""
|> buildCode 8000 None
|> Async.runWithTimeout 8000
|> Option.map (fun (fsxContent, errors) -> fsxContent, errors |> List.map fst)
|> _equal (
    Some (
        None,
        ["Cannot apply a forall with a term."]
    )
)

00:00:16 #81 [Debug] createTempDirectory / tempFolder: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230924-1118-1447-4761-4c7c441c8362 / result: { CreationTime = 2023-09-24 11:18:14 AM
  Exists = true }
00:00:16 #82 [Debug] runWithTimeoutAsync / timeout: 60
00:00:16 #83 [Debug] executeAsync / options: { Command =
   "dotnet "C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" port=13807"
  WorkingDirectory = None
  CancellationToken = Some System.Threading.CancellationToken
  OnLine = Some <fun:buildCode@3-920> }
00:00:16 #84 [Debug] > Server bound to: tcp://*:13807 & tcp://*:13808
00:00:16 #85 [Debug] buildFile / fsxContent:  / errors: [] / firstErrorMs:
00:00:16 #86 [Debug] > pwd: C:\home\git\polyglot\apps\spiral
00:00:16 #87 [Debug] > dll_path: C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release
00:00:16 #88 [Debug] sendJson / port: 13807 / json: {"

In [ ]:
//// test

"""
inl main () =
    real
        inl real_unbox forall a. (obj : a) : a =
            typecase obj with
            | _ => obj
        real_unbox `i32 1
"""
|> buildCode 8000 None
|> Async.runWithTimeout 8000
|> Option.map (fun (fsxContent, errors) -> fsxContent, errors |> List.map fst)
|> _equal (
    Some (
        None,
        ["The main function should not have a forall."]
    )
)

00:00:18 #93 [Debug] createTempDirectory / tempFolder: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230924-1118-1679-7995-7bef2bd92677 / result: { CreationTime = 2023-09-24 11:18:16 AM
  Exists = true }
00:00:18 #94 [Debug] runWithTimeoutAsync / timeout: 60
00:00:18 #95 [Debug] executeAsync / options: { Command =
   "dotnet "C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" port=13807"
  WorkingDirectory = None
  CancellationToken = Some System.Threading.CancellationToken
  OnLine = Some <fun:buildCode@3-920> }
00:00:19 #96 [Debug] > Server bound to: tcp://*:13807 & tcp://*:13808
00:00:19 #97 [Debug] buildFile / fsxContent:  / errors: [] / firstErrorMs:
00:00:19 #98 [Debug] > pwd: C:\home\git\polyglot\apps\spiral
00:00:19 #99 [Debug] > dll_path: C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release
00:00:19 #100 [Debug] sendJson / port: 13807 / json: {

## getFileTokenRange

In [ ]:
let inline getFileTokenRange timeout cancellationToken path = async {
    let fullPath = path |> System.IO.Path.GetFullPath
    let! code = fullPath |> FileSystem.readAllTextAsync
    let lines = code |> String.split [| '\n' |]

    let port =
        if fullPath |> String.startsWith (System.IO.Path.GetTempPath ())
            && fullPath |> String.contains "Microsoft.DotNet.Interactive.App" |> not
        then 13807
        else 13805

    let! serverPort, outputs, disposable = awaitCompiler port cancellationToken
    use _ = disposable

    let fileOpenObj = {| FileOpen = {| uri = fullPath |> getFileUri; spiText = code |} |}
    let! fileOpenResult = fileOpenObj |> sendObj serverPort

    let fileTokenRangeObj =
        {|
            FileTokenRange =
                {|
                    uri = fullPath |> getFileUri
                    range =
                        [|
                            {| line = 0; character = 0 |}
                            {| line = lines.Length - 1; character = lines.[lines.Length - 1].Length |}
                        |]
                |}
        |}
    let! fileTokenRangeResult = fileTokenRangeObj |> sendObj serverPort

    return fileTokenRangeResult |> Option.map FSharp.Json.Json.deserialize<int array>
}

## getCodeTokenRange

In [ ]:
let inline getCodeTokenRange timeout cancellationToken code = async {
    let! mainPath, disposable = persistCode timeout cancellationToken code
    use _ = disposable
    return! mainPath |> getFileTokenRange timeout cancellationToken
}

In [ ]:
//// test

"""inl main () = ()"""
|> getCodeTokenRange 8000 None
|> Async.runWithTimeout 8000
|> Option.flatten
|> _equal (Some [|0; 0; 3; 7; 0; 0; 4; 4; 0; 0; 0; 5; 1; 8; 0; 0; 1; 1; 8; 0; 0; 2; 1; 4; 0; 0;
2; 1; 8; 0; 0; 1; 1; 8; 0|])

00:00:28 #105 [Debug] createTempDirectory / tempFolder: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230924-1118-2662-6251-6a8dcd973cff / result: { CreationTime = 2023-09-24 11:18:26 AM
  Exists = true }
00:00:28 #106 [Debug] runWithTimeoutAsync / timeout: 60
00:00:28 #107 [Debug] executeAsync / options: { Command =
   "dotnet "C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" port=13807"
  WorkingDirectory = None
  CancellationToken = Some System.Threading.CancellationToken
  OnLine = Some <fun:it@4-344> }
00:00:28 #108 [Debug] > Server bound to: tcp://*:13807 & tcp://*:13808
00:00:28 #109 [Debug] > pwd: C:\home\git\polyglot\apps\spiral
00:00:28 #110 [Debug] > dll_path: C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release
00:00:29 #111 [Debug] sendJson / port: 13807 / json: {"FileOpen":{"spiText":"inl main () = ()","uri":"file:///C:\\Users\\i574n\\Ap

In [ ]:
//// test

"""inl main () = 1i32"""
|> getCodeTokenRange 8000 None
|> Async.runWithTimeout 8000
|> Option.flatten
|> _equal (Some [|0; 0; 3; 7; 0; 0; 4; 4; 0; 0; 0; 5; 1; 8; 0; 0; 1; 1; 8; 0; 0; 2; 1; 4; 0; 0;
2; 1; 3; 0; 0; 1; 3; 12; 0|])

00:00:32 #113 [Debug] createTempDirectory / tempFolder: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230924-1118-3067-6791-67bb5c7f2368 / result: { CreationTime = 2023-09-24 11:18:30 AM
  Exists = true }
00:00:32 #114 [Debug] runWithTimeoutAsync / timeout: 60
00:00:32 #115 [Debug] executeAsync / options: { Command =
   "dotnet "C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" port=13807"
  WorkingDirectory = None
  CancellationToken = Some System.Threading.CancellationToken
  OnLine = Some <fun:it@4-627> }
00:00:32 #116 [Debug] > Server bound to: tcp://*:13807 & tcp://*:13808
00:00:32 #117 [Debug] > pwd: C:\home\git\polyglot\apps\spiral
00:00:32 #118 [Debug] > dll_path: C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release
00:00:32 #119 [Debug] sendJson / port: 13807 / json: {"FileOpen":{"spiText":"inl main () = 1i32","uri":"file:///C:\\Users\\i574n\\

## Arguments

In [ ]:
[<RequireQualifiedAccess>]
type Arguments =
    | [<Argu.ArguAttributes.Mandatory>] BuildFile of string * string
    | Timeout of int

    interface Argu.IArgParserTemplate with
        member s.Usage =
            match s with
            | BuildFile _ -> nameof Arguments.BuildFile
            | Timeout _ -> nameof Arguments.Timeout

## main

In [ ]:
let main args =
    let argsMap = args |> Runtime.parseArgsMap<Arguments>

    let inputPath, outputPath =
        match argsMap.[nameof Arguments.BuildFile] with
        | [ Arguments.BuildFile (inputPath, outputPath) ] -> Some (inputPath, outputPath)
        | _ -> None
        |> Option.get

    let timeout =
        match argsMap |> Map.tryFind (nameof Arguments.Timeout) with
        | Some [ Arguments.Timeout timeout ] -> timeout
        | _ -> 30000

    async {
        let! outputCode, errors = inputPath |> buildFile timeout None
        
        errors
        |> List.map snd
        |> List.iter (fun error ->
            trace Critical (fun () -> $"main / error: {error}") getLocals
        )

        match outputCode with
        | Some outputCode ->
            do! outputCode |> FileSystem.writeAllTextAsync outputPath
            return 0
        | None ->
            return 1
    }
    |> Async.runWithTimeout timeout
    |> Option.defaultValue 1

In [ ]:
//// test

let args =
    System.Environment.GetEnvironmentVariable "ARGS"
    |> Runtime.splitArgs
    |> Seq.toArray

match args with
| [||] -> 0
| args -> if main args = 0 then 0 else failwith "main failed"

0